In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from pysentimiento.preprocessing import preprocess_tweet

df = pd.read_table("https://raw.githubusercontent.com/eliabisconti/haspeede/master/Dataset/haspeede2_dev_taskAB.tsv")

df["split"] = "train"

df_test = pd.read_table("https://raw.githubusercontent.com/collab-uniba/HateSpeechDetection_ITA/70d0b4240bb373c0e0a10f8a09abdc4d91f1b98e/Test%20sui%20Tweet/Train%20e%20test%20set%20UmBERTo/haspeede2_reference_taskAB-tweets.csv", header=None, names=["id", "text", "hs", "stereotype"])

df_test["split"] = "test"

df.columns = [t.strip() for t in df.columns]
df_test.columns = [t.strip() for t in df_test.columns]

df = pd.concat([df, df_test], axis=0)


#  These are the original tweets, but they lack the labels
df_true = pd.read_csv("../../data/hate/haspeede2020.csv")

#df_true = df_true[df_true["set"].isin(["training", "test_tw"])].copy()

df_true["text"] = df_true["text_original"].apply(
    lambda x: preprocess_tweet(x, lang="it", user_token="@user", demoji=False, preprocess_hashtags=False))


In [2]:
from thefuzz import fuzz
from tqdm.auto import tqdm
# Find the most similar tweet in df_test["text"]

best_indices = []
original_text = []
num_errors = 0

limit = 50

file = open("log.txt", "w")

for idx, row in tqdm(df.iterrows(), total=len(df)):
    # Find the most similar tweet in df_test["text"]
    text = row["text"][:limit]

    ratios = []

    for idx2, row2 in df_true.iterrows():
        ratio = fuzz.partial_ratio(row2["text"][:limit], text)
        ratios.append((idx2, ratio, row2["text_original"]))
        if ratio == 100:
            break

    ratios = sorted(ratios, key=lambda x: x[1], reverse=True)
    best_idx, ratio, new_text = ratios[0]

    if ratio > 85:
        best_indices.append(best_idx)
        original_text.append(new_text)
        # Remove from df_true to make it faster
        df_true = df_true.drop(best_idx)
        print("="*80, file=file)
        print(f'{ratio} -- {row["text"]} -> {new_text}', file=file)
    else:
        best_indices.append(None)
        original_text.append(None)
        print(num_errors)
        print(f"Could not find a similar tweet for\n{row['text']}")
        print(f"Preprocessed: {text}")
        print(f"Most similar: {new_text}")
        print(f"Ratio: {ratio}")
        print("=" * 80)
        num_errors += 1
        

  0%|          | 0/8100 [00:00<?, ?it/s]

0
Could not find a similar tweet for
+++#Siria🇸🇾 Evacuati civili dalla città terroristi #IS avanzano dopo violenti scontri con Esercito #siriano a #Palmira ma non è finita+++ 
Preprocessed: +++#Siria🇸🇾 Evacuati civili dalla città terroristi
Most similar: La via italiana alla lotta al terrorismo di matrice islamica https://t.co/qb6Rkqk9jS
Ratio: 59
1
Could not find a similar tweet for
Lavori da #rom URL 
Preprocessed: Lavori da #rom URL 
Most similar: Lavori da #rom https://t.co/YEBMTbhsDQ
Ratio: 79
2
Could not find a similar tweet for
Roma, nascerà un nuovo campo rom URL URL 
Preprocessed: Roma, nascerà un nuovo campo rom URL URL 
Most similar: Roma, nascerà un nuovo campo rom https://t.co/HxFGhTUE5n - Adriano Montanaro Alessandria
Ratio: 85
3
Could not find a similar tweet for
Studentessa cinese morta a #Roma condannati i due nomadi URL "condannati" a rimanere liberi grazie alle depenalizzazioni 
Preprocessed: Studentessa cinese morta a #Roma condannati i due 
Most similar: Studentess

In [3]:
df["original_text"] = original_text

# Count na values

df["original_text"].isna().sum()

63

In [4]:
df_true.shape

(566, 5)

In [5]:
num_errors = 0
for idx, row in tqdm(df.iterrows(), total=len(df)):
    # Find the most similar tweet in df_test["text"]
    if pd.notna(row["original_text"]):
        continue
    text = row["text"][:limit]

    ratios = []

    for idx2, row2 in df_true.iterrows():
        ratio = fuzz.partial_ratio(row2["text"][:limit], text)
        ratios.append((idx2, ratio, row2["text"]))
        if ratio == 100:
            break

    ratios = sorted(ratios, key=lambda x: x[1], reverse=True)
    best_idx, ratio, new_text = ratios[0]

    if ratio > 55:
        # Remove from df_true to make it faster
        df_true = df_true.drop(best_idx)

        df.loc[idx, "original_text"] = new_text
   

  0%|          | 0/8100 [00:00<?, ?it/s]

In [6]:
df["original_text"].isna().sum()

4

In [7]:
df.to_csv("haspeede2020.csv", index=False)

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd

df = pd.read_csv("haspeede2020.csv")

In [2]:
pd.options.display.max_colwidth = 1000

df

,id,text,hs,stereotype,split,original_text
0,2066,"È terrorismo anche questo, per mettere in uno stato di soggezione le persone e renderle innocue, mentre qualcuno... URL",0,0,train,"È terrorismo anche questo, per mettere in uno stato di soggezione le persone e renderle innocue, mentre qualcuno... https://t.co/GFAvh0QLe5"
1,2045,@user @user infatti finché ci hanno guadagnato con i campi #rom tutto era ok con #Alemanno #Ipocriti,0,0,train,@Nanoalto @FedeAngeli infatti finché ci hanno guadagnato con i campi #rom tutto era ok con #Alemanno #Ipocriti
2,61,"Corriere: Tangenti, Mafia Capitale dimenticataMazzette su buche e campi rom URL #roma",0,0,train,"Corriere: Tangenti, Mafia Capitale dimenticataMazzette su buche e campi rom https://t.co/RTPu3LNTzF #roma"
3,1259,"@user ad uno ad uno, perché quando i migranti israeliti arrivarono in terra di Canaan fecero fuori tutti i Canaaniti.",0,0,train,"@Pontifex_it ad uno ad uno, perché quando i migranti israeliti arrivarono in terra di Canaan fecero fuori tutti i Canaaniti."
4,949,Il divertimento del giorno? Trovare i patrioti italiani che inneggiano contro i rom facendo la spesa alla #Lidl (multinazionale tedesca).,0,0,train,Il divertimento del giorno? Trovare i patrioti italiani che inneggiano contro i rom facendo la spesa alla #Lidl (multinazionale tedesca).
...,...,...,...,...,...,...
8095,10091,"Il Razzismo gli viene inculcato sin da bambini dal corano e dall'islam Cani, Maiali. Cristiani ed Ebrei sono impuri, infedeli e da sterminare. Il nemico e gran Pezzo di Merda è colui che li vuole qui a tutti i costi.",1,1,test,"Il Razzismo gli viene inculcato sin da bambini dal corano e dall'islam Cani, Maiali. Cristiani ed Ebrei sono impuri, infedeli e da sterminare. Il nemico e gran Pezzo di Merda è colui che li vuole qui a tutti i costi."
8096,10046,"PERCHÉ DI QUESTO ORRENDO STUPRO NON DI PARLA? INFORMAZIONE CORROTTA,LA SINISTRA CONTINUA A COPRIRE I DELINQUENTI STRANIERI !! #viterbo #casapound #migranti URL",1,1,test,"PERCHÉ DI QUESTO ORRENDO STUPRO NON DI PARLA? INFORMAZIONE CORROTTA,LA SINISTRA CONTINUA A COPRIRE I DELINQUENTI STRANIERI !! #viterbo #casapound #migranti https://t.co/mcCMiyJJFN"
8097,10377,"FERMIAMO L'IMPOSTORE. Forse saranno necesarie altre CROCIATE, visto che il fiume di fango non si ferma.Anzi Vaticano, SX e POTERI MASSONICI ci stanno massacrando. Quando mai in Italia qualcuno veniva SGOZZATO, PRESO A SPRANGATE, FATTO A PEZZI? Fermiamo l'invasione dell'ISLAM!!!",1,1,test,"FERMIAMO L'IMPOSTORE. Forse saranno necesarie altre CROCIATE, visto che il fiume di fango non si ferma.Anzi Vaticano, SX e POTERI MASSONICI ci stanno massacrando. Quando mai in Italia qualcuno veniva SGOZZATO, PRESO A SPRANGATE, FATTO A PEZZI? Fermiamo l'invasione dell'ISLAM!!!"
8098,10199,@user @user @user @user @user @user @user @user @user @user Matteo non si tocca...fuori i delinquenti stranieri da casa nostra !,1,1,test,@rep_bologna @beppe_grillo\n@AlfonsoBonafede\n@Mov5Stelle\n@DaniloToninelli\n@Roberto_Fico\n@Eli_Trenta\n@GiuliaGrilloM5S\n@c_appendino\n@virginiaraggi\n@nogarin\n@LuigiGallo15\n@BarbaraLezzi\n@ANMagistrati\n@luigidimaio\n@GiuseppeConteIT rispedire a casa loro criminali stranieri no vero? Solo costi!!!


## Sanity check


In [2]:
#check that meanings with sentence bert are more or less the same
import numpy as np
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm
from pysentimiento.preprocessing import preprocess_tweet

model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

num_errors = 0
for idx, row in tqdm(df.iterrows(), total=len(df)):
    if pd.isna(row["original_text"]):
        continue
    text = row["text"]
    original_text = preprocess_tweet(row["original_text"], lang="it", user_token="@user", url_token="URL", demoji=False, preprocess_hashtags=False)

    text_emb = model.encode(text)
    original_text_emb = model.encode(original_text)
    
    # Check cosine similarity
    cos_sim = np.dot(text_emb, original_text_emb) / (np.linalg.norm(text_emb) * np.linalg.norm(original_text_emb))

    df.loc[idx, "cos_sim"] = cos_sim




  0%|          | 0/8100 [00:00<?, ?it/s]

In [3]:
df.to_csv("haspeede2020.cossim.csv", index=False)

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd

df = pd.read_csv("haspeede2020.cossim.csv")

In [7]:
# Count less than 0.5

possible_errors = df[df["cos_sim"] < 0.75].sort_values("cos_sim", ascending=False)

for idx, row in possible_errors.iterrows():
    print(row["text"])
    print(row["original_text"])
    print(row["cos_sim"])
    print("="*80)

Business rom: nel mirino della Procura coop e politici URL URL 
Business rom: nel mirino della Procura coop e politici https://t.co/mZ5W8AIRvh - Adriano Montanaro Alessandria
0.7423011660575867
Business rom: nel mirino della Procura coop e politici URL - Adriano Montanaro Alessandria 
Business rom: nel mirino della Procura coop e politici #newsrss24italia https://t.co/KLq65B2KvL
0.7373237013816833
Il rom che derubò Ermes dà fuoco alla sua roulotte per ottenere una casa sto figlio di zoccola ha capito bene cosa fare x sistemarsi 
Il rom che derubò Ermes dà fuoco alla sua roulotte per ottenere una casa #Generale https://t.co/k9DVPcNf85
0.701755166053772
Il rom che derubò Ermes dà fuoco alla sua roulotte per ottenere una casa #Generale URL 
Il rom che derubò Ermes dà fuoco alla sua roulotte per ottenere una casa sto figlio di zoccola ha capito bene cosa fare x sistemarsi
0.701755166053772
Accordo Italia-Libia sui migranti, i punti salienti: Da controllo confini a finanziamento centri acco

In [20]:
df_ok = df[df["cos_sim"] >= 0.71].copy()

In [21]:
df_ok["split"].value_counts()

train    6797
test     1238
Name: split, dtype: int64

In [22]:
df["split"].value_counts()

train    6837
test     1263
Name: split, dtype: int64

In [23]:
# drop text column and cossim
df_ok = df_ok.drop("text", axis=1)
df_ok = df_ok.drop("cos_sim", axis=1)
# Rename original_text to text
df_ok = df_ok.rename(columns={"original_text": "text"})

In [27]:
from sklearn.model_selection import train_test_split


df_train = df_ok[df_ok["split"] == "train"].copy()
df_test = df_ok[df_ok["split"] == "test"].copy()

train_df, dev_df = train_test_split(
    df_ok, test_size=0.18, random_state=2022)

# drop split column
train_df = train_df.drop("split", axis=1)
dev_df = dev_df.drop("split", axis=1)
df_test = df_test.drop("split", axis=1)


train_df.shape, dev_df.shape, df_test.shape

((6588, 4), (1447, 4), (1238, 4))

In [28]:
pd.options.display.max_colwidth = 200
train_df.columns, dev_df.columns, df_test.columns

(Index(['id', 'hs', 'stereotype', 'text'], dtype='object'),
 Index(['id', 'hs', 'stereotype', 'text'], dtype='object'),
 Index(['id', 'hs', 'stereotype', 'text'], dtype='object'))

In [29]:
from datasets import DatasetDict, Dataset, Value, Features, ClassLabel

features = Features({
    'id': Value('string'),
    'text': Value('string'),
    'hs': ClassLabel(num_classes=2, names=["non hateful", "hateful"]),
    "stereotype": ClassLabel(num_classes=2, names=["non stereotype", "stereotype"]),
})

train = Dataset.from_pandas(train_df, features=features, preserve_index=False)
dev = Dataset.from_pandas(dev_df, features=features, preserve_index=False)
test = Dataset.from_pandas(df_test, features=features, preserve_index=False)

ds = DatasetDict(
    train=train,
    dev=dev,
    test=test
)

In [30]:
ds.push_to_hub("pysentimiento/it_haspeede", private=True)

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

...

Pushing split test to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

## Check Haspeede 2020

In [10]:
from datasets import load_dataset

ds = load_dataset("pysentimiento/it_haspeede")

ds

Using custom data configuration pysentimiento--it_haspeede-213526540a343f67
Found cached dataset parquet (/users/jmperez/.cache/huggingface/datasets/pysentimiento___parquet/pysentimiento--it_haspeede-213526540a343f67/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    dev: Dataset({
        features: ['id', 'text', 'hs', 'stereotype'],
        num_rows: 1231
    })
    train: Dataset({
        features: ['id', 'text', 'hs', 'stereotype'],
        num_rows: 5606
    })
})

Could not find a similar tweet for #ERROR!
Most similar: E L'IMAM FRANCESCO CONTINUA A SOSTENERE CHE GLI IMMIGRATI NON SONO UN PROBLEMA.  URL


In [16]:
training_df = df[df["set"] == "train"]

In [17]:
len(set(training_df["text_original"]) - set(text_ds))

0

In [21]:
test_df = df[df["set"] == "test_tw"]

In [24]:
test_df["text_original"].iloc[0]

'@LaStampaTV @LaStampa i musulmano ammazzano tutti quelli che per il loro fumoso cervello sono "infedeli". i nostri terroristi istituzionali ci obbligano ad accogliere e mantenere i nostri assassini'

In [32]:
pd.options.display.max_colwidth = 100

df_test

,text,hs,stereotype
id,,,
11834,@user A me pare una scelta politica suicida puntare tutto su una battaglia sicuramente perdente ...,1,0
12113,"@user e' un perfetto musulmano!!! chi dice il contrario mente, applica alla lettera la sura ""la ...",1,1
11770,Mai Tg e i giornaloni hanno parlato di questa ennesima porcata fatta da immigrati ?? ⤵️⤵️⤵️⤵️ S...,1,1
11937,@user Ipocriti farabutti. Fanno morire i terremotati per I Bastardi clandestini immigrati schifosi.,1,1
11870,@user @user @user L'IMMIGRAZIONE C'E' STATO UN CALO FISIOLOGICO QUANTI RINPATRII HA FATTO ? CI S...,1,1
...,...,...,...
10091,"Il Razzismo gli viene inculcato sin da bambini dal corano e dall'islam Cani, Maiali. Cristiani e...",1,1
10046,"PERCHÉ DI QUESTO ORRENDO STUPRO NON DI PARLA? INFORMAZIONE CORROTTA,LA SINISTRA CONTINUA A COPRI...",1,1
10377,"FERMIAMO L'IMPOSTORE. Forse saranno necesarie altre CROCIATE, visto che il fiume di fango non si...",1,1


In [ ]:
df_t